In [1]:
import os
import pandas as pd
import numpy as np
import mygene
import rpy2.robjects as ro
#import stringdb
import openpyxl
#import PyWGCNA
import xlsxwriter
import matplotlib.pyplot as plt

from scipy.stats import spearmanr
from scipy.stats import linregress
from rpy2.rinterface_lib.embedded import RRuntimeError
from rpy2.robjects import pandas2ri, Formula
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
from statsmodels.stats.multitest import multipletests
from functools import reduce
from openpyxl import load_workbook
from collections import Counter
import warnings

# suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
base = importr('base')
stats = importr('stats')
limma = importr('limma')
writexl = importr('writexl')

In [3]:
root_dir = "/Volumes/Expansion/Thesis Work/Datasets/GDSC/Expression/"
response_dir = "/Volumes/Expansion/Thesis Work/Results/preprocessed_results2/response/"
save_results_to = "/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/"
gdsc_ensembl_expr_file = "/Volumes/Expansion/Thesis Work/Datasets/GDSC/Expression/GDSC_micro.BrainArray.RMAlog2Average.ENSEMBL.Expr_renamed.tsv"

In [4]:
gdsc_expr = pd.read_csv(root_dir + "GDSC_micro.BrainArray.RMAlog2Average.ENSEMBL.Expr_renamed.tsv",
                       index_col = 0,
                       sep = "\t",
                       decimal=",")

GDSCE = pd.DataFrame.transpose(gdsc_expr)
GDSCE.index.name = "sample_name"
GDSCE = GDSCE.map(pd.to_numeric) 
GDSCE.index = GDSCE.index.astype('int')

GDSCE.head()

ensembl_gene_id,ENSG00000121410,ENSG00000175899,ENSG00000171428,ENSG00000156006,ENSG00000196136,ENSG00000114771,ENSG00000127837,ENSG00000129673,ENSG00000090861,ENSG00000183044,...,ENSG00000225050,ENSG00000272808,ENSG00000268480,ENSG00000142396,ENSG00000276184,ENSG00000254473,ENSG00000249382,ENSG00000231131,ENSG00000271360,ENSG00000249618
sample_name,,,,,,,,,,,,,,,,,,,,,
683665,3.805253,3.655408,7.643880,3.039498,3.119811,2.963500,7.724297,2.995156,8.036325,3.229925,...,2.741542,3.707337,3.268119,9.517192,3.049383,4.164326,2.689977,3.266312,2.981273,3.154424
683667,5.140331,3.612306,4.377345,2.958211,3.642967,2.878635,8.262489,2.994176,9.085753,3.964241,...,3.203552,3.155934,3.138888,9.001884,3.158700,3.942120,2.739765,2.857805,3.009378,3.480829
684052,4.252704,3.337473,5.563119,3.157324,2.969843,2.927023,8.209935,2.904290,9.638053,5.612448,...,2.818725,3.405444,3.119432,8.986123,2.591639,4.989951,2.589481,2.809062,3.045186,3.481143
684055,5.594708,3.503549,6.489859,3.114843,2.933490,2.786126,8.336650,2.504999,10.387287,5.925628,...,2.791858,3.297908,2.981723,9.286065,2.811643,4.453397,2.590231,2.659379,2.759121,3.520399
684057,4.903946,6.705561,5.722173,3.202469,3.153036,3.113733,8.586205,2.910457,8.545740,3.586090,...,2.783178,3.126472,3.201917,8.951428,2.723362,4.390062,2.827584,2.950365,3.100417,3.582252


In [5]:
gdsc_expr = dict()
# for dir in os.listdir(save_results_to): 

for dir in ["MITOSISi","CYTOi","DNA_REPi","EGFRi"]: 
     
    gdsc_expr[dir] = pd.read_csv(response_dir + "GDSC_response." + dir + ".tsv",
                                 sep = "\t",
                                 index_col = 0,
                                 decimal = ",")
    
    gdsc_expr[dir] = pd.DataFrame(gdsc_expr[dir])
    gdsc_expr[dir].index.name = "sample_name"
    gdsc_expr[dir].index = gdsc_expr[dir].index.astype('int') 
 
    #print("GDSC_response." + dir + " Expression data")
    #print(gdsc_expr[dir].head())
    #print("\n")
 
    drugs_inhibitor = np.unique(gdsc_expr[dir]["drug"])
    print("GDSC " + dir + " inhibitors:", *drugs_inhibitor)
    print("\n") 
 
    gdsc_drugs = dict()
    for drug in drugs_inhibitor:
        gdsc_drugs[drug] = gdsc_expr[dir].loc[gdsc_expr[dir]["drug"] == drug,]
        gdsc_drugs[drug].index.name = "sample_name" 
 
     
    GDSCE_dfs = dict()
    GDSC_drugs_dfs = dict()
    for dat_frame in gdsc_drugs:
        gdsc_drugs[dat_frame] = gdsc_drugs[dat_frame].loc[~gdsc_drugs[dat_frame]["response"].isnull()]
        exprs_response =set(GDSCE.index).intersection(set(gdsc_drugs[dat_frame].index))
    
        GDSCE_dfs[dat_frame] = GDSCE.loc[list(exprs_response),]
        GDSC_drugs_dfs[dat_frame] = gdsc_drugs[dat_frame].loc[list(exprs_response),]
        GDSC_drugs_dfs[dat_frame] = GDSC_drugs_dfs[dat_frame].loc[~GDSC_drugs_dfs[dat_frame]["response"].isnull()]
 
        os.chdir(save_results_to + dir)
        os.makedirs(dat_frame, exist_ok = True)
        
        pd.DataFrame(GDSC_drugs_dfs[dat_frame]["response"]).to_csv(save_results_to + dir + "/" + dat_frame + "/gdsc_" + dat_frame + "_sample_info.tsv",
                                                            sep ="\t")
        
        print("Total " + dat_frame + " samples:", GDSC_drugs_dfs[dat_frame].shape[0])
    
        #gdsc_drug_dfs[df] = pd.merge(GDSCE, gdsc_drugs[df]["response"], left_index=True, right_index=True)
        
        filter_R = GDSC_drugs_dfs[dat_frame]["response"] == "R"
        resistant = GDSC_drugs_dfs[dat_frame].loc[filter_R,]
        
        filter_S = GDSC_drugs_dfs[dat_frame]["response"] == "S"
        sensitive = GDSC_drugs_dfs[dat_frame].loc[filter_S,]
        
        print(f'R:{resistant.shape[0]}, S:{sensitive.shape[0]}\n')
        
        # Convert data and design pandas dataframes to R dataframes
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_data = ro.conversion.py2rpy(pd.DataFrame.transpose(GDSCE_dfs[dat_frame]).map(pd.to_numeric))
            r_design = ro.conversion.py2rpy(pd.DataFrame(GDSC_drugs_dfs[dat_frame]))
            # Use the genes index column from data as a R String Vector
            genes = ro.StrVector(
                [
                    str(index)
                    # added tovalues to convert to numpy array
                    for index in pd.DataFrame.transpose(GDSCE_dfs[dat_frame]).index.tolist()
                    # for index in data.index.tolist()
                ]
        )
        
        # Create a model matrix using design's Target column using the R formula "~0 + f" 
        # to get all the unique factors as columns
        
        f = base.factor(r_design.rx2('response'),
                        levels=base.unique(r_design.rx2('response')))
        form = Formula('~0 + f')
        form.environment['f'] = f
        r_design = stats.model_matrix(form)
        r_design.colnames = base.levels(f)
        
        # Fit the data to the design using lmFit from limma
        fit = limma.lmFit(r_data, r_design)
        
        # Make a contrasts matrix with the 1st and the last unique values
        contrast_matrix = limma.makeContrasts(
            f"{r_design.colnames[0]}-{r_design.colnames[-1]}", levels=r_design)
        
        # Fit the contrasts matrix to the lmFit data & calculate the bayesian fit
        fit2 = limma.contrasts_fit(fit, contrast_matrix)
        fit2 = limma.eBayes(fit2)
        
        # topTreat the bayesian fit using the contrasts and add the genelist
        r_output = limma.topTreat(fit2, coef=1, genelist=genes, number=np.Inf)
        
        writexl.write_xlsx(r_output, save_results_to + dir + "/" + dat_frame + "/Limma Results (EnsemblID).xlsx")
     
 
    dataframe = pd.DataFrame(list()) # empty dataframe
    dataframe.to_excel(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx')
 
    gdsc_pValues = dict()
    for limma_result in gdsc_drugs.keys():
        os.chdir(save_results_to + dir + "/" + limma_result)
        print(os.getcwd())
 
 
        gdsc_pValues[limma_result] = pd.read_excel("Limma Results (EnsemblID).xlsx",
                                                  index_col = 0,
                                                  decimal = ",")
        
        filter = (gdsc_pValues[limma_result]["adj.P.Val"] < 0.05) & (abs(gdsc_pValues[limma_result]["logFC"]) > 1)
        gdsc_pValues[limma_result] = gdsc_pValues[limma_result].loc[filter,]

        if gdsc_pValues[limma_result].loc[filter].shape[0] == 0:
            continue
            
        gene_symbol = gdsc_pValues[limma_result].index
        
        mg = mygene.MyGeneInfo()
        df = mg.getgenes(gene_symbol, 
                         email='alpsoy@tau.edu.tr', 
                         species=9606,
                         as_dataframe=True,
                         fetch_all = True)
        
        #df['symbol'].index = df['symbol'].index.astype(np.int64)
    
        gdsc_pValues[limma_result].index.name = "ensembl_gene_id"
        df['symbol'].index.name = "ensembl_gene_id"
        df['ensembl_gene_id'] = df.index
        gene_symbol_df = df[['ensembl_gene_id','symbol']] 
        
        gene_symbol_df.drop_duplicates(subset=["ensembl_gene_id"], inplace = True)
        gdsc_pValues[limma_result]['Gene.Symbol'] = gene_symbol_df['symbol']  
        
        with pd.ExcelWriter(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx', engine="openpyxl", if_sheet_exists = "replace", mode = 'a') as writer:
            gdsc_pValues[limma_result].to_excel(writer, sheet_name = limma_result)
        
        #print(limma_result)
        #print(gdsc_pValues[limma_result].head())
        
    workbook=openpyxl.load_workbook(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx')
    try:
        del workbook['Sheet1']
    except:
        print("Blank worksheet has already been removed")
    workbook.save(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx')

    
    diff_exp_genes = load_workbook(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx')
    
    DEGs = dict()
    deg_files = dict()
    for deg_file in gdsc_drugs.keys():
        if deg_file in diff_exp_genes.sheetnames:
            
             DEGs[deg_file] = pd.read_excel(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx',
                                            index_col = 0,
                                            decimal = ",",
                                            sheet_name = deg_file)
            
             DEGs[deg_file] = DEGs[deg_file].loc[DEGs[deg_file]["adj.P.Val"] < 0.05]

             deg_files[deg_file] = DEGs[deg_file]['Gene.Symbol']
             deg_files[deg_file] = set(deg_files[deg_file])
 
    gene_list = list()
    for deg in deg_files.values():
        gene_list = gene_list + list(deg)
    counter = Counter(gene_list)
    
    freq_dataframe = pd.DataFrame.transpose(pd.DataFrame([counter]))
    freq_dataframe.rename(columns = {0: 'Frequency'}, inplace = True)
    freq_dataframe.sort_values("Frequency", ascending = False, inplace = True)
    freq_dataframe.index.name = "Gene Symbol"
    filter = freq_dataframe["Frequency"] == max(freq_dataframe["Frequency"])
    freq_degs = freq_dataframe.loc[filter]
 
    print("\n")
    print(f'{dir}: Differentially expressed genes shared for most of the drugs ({max(freq_dataframe["Frequency"])} out of {len(deg_files.keys())}): {freq_degs.shape[0]}')
    print("\n")

    with pd.ExcelWriter(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx', engine="openpyxl", if_sheet_exists = "replace", mode = 'a') as writer:
        freq_dataframe.to_excel(writer, sheet_name = 'Common DEGs')  

GDSC MITOSISi inhibitors: BI-2536 GSK1070916 GW843682X Genentech Cpd 10 MPS-1-IN-1 NPK76-II-72-1 Paclitaxel S-Trityl-L-cysteine SB-715992 VX-680 ZM-447439


Total BI-2536 samples: 368
R:340, S:28

Total GSK1070916 samples: 747
R:611, S:136

Total GW843682X samples: 370
R:357, S:13

Total Genentech Cpd 10 samples: 771
R:685, S:86

Total MPS-1-IN-1 samples: 767
R:719, S:48

Total NPK76-II-72-1 samples: 769
R:689, S:80

Total Paclitaxel samples: 370
R:344, S:26

Total S-Trityl-L-cysteine samples: 369
R:354, S:15

Total SB-715992 samples: 770
R:745, S:25

Total VX-680 samples: 365
R:286, S:79

Total ZM-447439 samples: 694
R:628, S:66

/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/MITOSISi/BI-2536
/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/MITOSISi/GSK1070916
/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/MITOSISi/GW843682X
/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/MITOSISi/Genentech Cpd 10
/Volumes/Expansion/Thesis Work/Results/

In [6]:
gdsc_expr = dict()
for dir in ["MITOSISi","CYTOi","DNA_REPi","EGFRi"]: 
     
    gdsc_expr[dir] = pd.read_csv(response_dir + "GDSC_response." + dir + ".tsv",
                                 sep = "\t",
                                 index_col = 0,
                                 decimal = ",")
    
    gdsc_expr[dir] = pd.DataFrame(gdsc_expr[dir])
    gdsc_expr[dir].index.name = "sample_name"
    gdsc_expr[dir].index = gdsc_expr[dir].index.astype('int') 
 
    #print("GDSC_response." + dir + " Expression data")
    #print(gdsc_expr[dir].head())
    #print("\n")
 
    drugs_inhibitor = np.unique(gdsc_expr[dir]["drug"])
    print("GDSC " + dir + " inhibitors:", *drugs_inhibitor)
    print("\n") 
 
    gdsc_drugs = dict()
    for drug in drugs_inhibitor:
        gdsc_drugs[drug] = gdsc_expr[dir].loc[gdsc_expr[dir]["drug"] == drug,]
        gdsc_drugs[drug].index.name = "sample_name" 
 
     
    GDSCE_dfs = dict()
    GDSC_drugs_dfs = dict()
    for dat_frame in gdsc_drugs:
        gdsc_drugs[dat_frame] = gdsc_drugs[dat_frame].loc[~gdsc_drugs[dat_frame]["response"].isnull()]
        exprs_response =set(GDSCE.index).intersection(set(gdsc_drugs[dat_frame].index))
    
        GDSCE_dfs[dat_frame] = GDSCE.loc[list(exprs_response),]
        GDSC_drugs_dfs[dat_frame] = gdsc_drugs[dat_frame].loc[list(exprs_response),]
        GDSC_drugs_dfs[dat_frame] = GDSC_drugs_dfs[dat_frame].loc[~GDSC_drugs_dfs[dat_frame]["response"].isnull()]
 
        os.chdir(save_results_to + dir)
        os.makedirs(dat_frame, exist_ok = True)
        
        pd.DataFrame(GDSC_drugs_dfs[dat_frame]["response"]).to_csv(save_results_to + dir + "/" + dat_frame + "/gdsc_" + dat_frame + "_sample_info.tsv",
                                                            sep ="\t")
        
        print("Total " + dat_frame + " samples:", GDSC_drugs_dfs[dat_frame].shape[0])
    
        #gdsc_drug_dfs[df] = pd.merge(GDSCE, gdsc_drugs[df]["response"], left_index=True, right_index=True)
        
        filter_R = GDSC_drugs_dfs[dat_frame]["response"] == "R"
        resistant = GDSC_drugs_dfs[dat_frame].loc[filter_R,]
        
        filter_S = GDSC_drugs_dfs[dat_frame]["response"] == "S"
        sensitive = GDSC_drugs_dfs[dat_frame].loc[filter_S,]
        
        print(f'R:{resistant.shape[0]}, S:{sensitive.shape[0]}\n')
        
        # Convert data and design pandas dataframes to R dataframes
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_data = ro.conversion.py2rpy(pd.DataFrame.transpose(GDSCE_dfs[dat_frame]).map(pd.to_numeric))
            r_design = ro.conversion.py2rpy(pd.DataFrame(GDSC_drugs_dfs[dat_frame]))
            # Use the genes index column from data as a R String Vector
            genes = ro.StrVector(
                [
                    str(index)
                    # added tovalues to convert to numpy array
                    for index in pd.DataFrame.transpose(GDSCE_dfs[dat_frame]).index.tolist()
                    # for index in data.index.tolist()
                ]
        )
        
        # Create a model matrix using design's Target column using the R formula "~0 + f" 
        # to get all the unique factors as columns
        
        f = base.factor(r_design.rx2('response'),
                        levels=base.unique(r_design.rx2('response')))
        form = Formula('~0 + f')
        form.environment['f'] = f
        r_design = stats.model_matrix(form)
        r_design.colnames = base.levels(f)
        
        # Fit the data to the design using lmFit from limma
        fit = limma.lmFit(r_data, r_design)
        
        # Make a contrasts matrix with the 1st and the last unique values
        contrast_matrix = limma.makeContrasts(
            f"{r_design.colnames[0]}-{r_design.colnames[-1]}", levels=r_design)
        
        # Fit the contrasts matrix to the lmFit data & calculate the bayesian fit
        fit2 = limma.contrasts_fit(fit, contrast_matrix)
        fit2 = limma.eBayes(fit2)
        
        # topTreat the bayesian fit using the contrasts and add the genelist
        r_output = limma.topTreat(fit2, coef=1, genelist=genes, number=np.Inf)
        
        writexl.write_xlsx(r_output, save_results_to + dir + "/" + dat_frame + "/Limma Results (EnsemblID).xlsx")
     
 
    dataframe = pd.DataFrame(list()) # empty dataframe
    dataframe.to_excel(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx')
 
    gdsc_pValues = dict()
    for limma_result in gdsc_drugs.keys():
        os.chdir(save_results_to + dir + "/" + limma_result)
        print(os.getcwd())
 
 
        gdsc_pValues[limma_result] = pd.read_excel("Limma Results (EnsemblID).xlsx",
                                                  index_col = 0,
                                                  decimal = ",")
        
        filter = gdsc_pValues[limma_result]["adj.P.Val"] < 0.05
        gdsc_pValues[limma_result] = gdsc_pValues[limma_result].loc[filter,]

        if gdsc_pValues[limma_result].loc[gdsc_pValues[limma_result]["adj.P.Val"] < 0.05].shape[0] == 0:
            continue
            
        gene_symbol = gdsc_pValues[limma_result].index
        
        mg = mygene.MyGeneInfo()
        df = mg.getgenes(gene_symbol, 
                         email='alpsoy@tau.edu.tr', 
                         species=9606,
                         as_dataframe=True,
                         fetch_all = True)
        
        #df['symbol'].index = df['symbol'].index.astype(np.int64)
    
        gdsc_pValues[limma_result].index.name = "ensembl_gene_id"
        df['symbol'].index.name = "ensembl_gene_id"
        df['ensembl_gene_id'] = df.index
        gene_symbol_df = df[['ensembl_gene_id','symbol']] 
        
        gene_symbol_df.drop_duplicates(subset=["ensembl_gene_id"], inplace = True)
        gdsc_pValues[limma_result]['Gene.Symbol'] = gene_symbol_df['symbol']  
        
        with pd.ExcelWriter(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx', engine="openpyxl", if_sheet_exists = "replace", mode = 'a') as writer:
            gdsc_pValues[limma_result].to_excel(writer, sheet_name = limma_result)
        
        #print(limma_result)
        #print(gdsc_pValues[limma_result].head())
        
    workbook=openpyxl.load_workbook(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx')
    try:
        del workbook['Sheet1']
    except:
        print("Blank worksheet has already been removed")
    workbook.save(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx')

    
    diff_exp_genes = load_workbook(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx')
    
    DEGs = dict()
    deg_files = dict()
    for deg_file in gdsc_drugs.keys():
        if deg_file in diff_exp_genes.sheetnames:
            
             DEGs[deg_file] = pd.read_excel(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx',
                                            index_col = 0,
                                            decimal = ",",
                                            sheet_name = deg_file)
            
             DEGs[deg_file] = DEGs[deg_file].loc[DEGs[deg_file]["adj.P.Val"] < 0.05]

             deg_files[deg_file] = DEGs[deg_file]['Gene.Symbol']
             deg_files[deg_file] = set(deg_files[deg_file])
 
    gene_list = list()
    for deg in deg_files.values():
        gene_list = gene_list + list(deg)
    counter = Counter(gene_list)
    
    freq_dataframe = pd.DataFrame.transpose(pd.DataFrame([counter]))
    freq_dataframe.rename(columns = {0: 'Frequency'}, inplace = True)
    freq_dataframe.sort_values("Frequency", ascending = False, inplace = True)
    freq_dataframe.index.name = "Gene Symbol"
    filter = freq_dataframe["Frequency"] == max(freq_dataframe["Frequency"])
    freq_degs = freq_dataframe.loc[filter]
 
    print("\n")
    print(f'{dir}: Differentially expressed genes shared for most of the drugs ({max(freq_dataframe["Frequency"])} out of {len(deg_files.keys())}): {freq_degs.shape[0]}')
    print("\n")

    with pd.ExcelWriter(save_results_to + dir + f'/{dir}_Differentially_Expressed_Genes (EnsemblID).xlsx', engine="openpyxl", if_sheet_exists = "replace", mode = 'a') as writer:
        freq_dataframe.to_excel(writer, sheet_name = 'Common DEGs')  



GDSC MITOSISi inhibitors: BI-2536 GSK1070916 GW843682X Genentech Cpd 10 MPS-1-IN-1 NPK76-II-72-1 Paclitaxel S-Trityl-L-cysteine SB-715992 VX-680 ZM-447439


Total BI-2536 samples: 368
R:340, S:28

Total GSK1070916 samples: 747
R:611, S:136

Total GW843682X samples: 370
R:357, S:13

Total Genentech Cpd 10 samples: 771
R:685, S:86

Total MPS-1-IN-1 samples: 767
R:719, S:48

Total NPK76-II-72-1 samples: 769
R:689, S:80

Total Paclitaxel samples: 370
R:344, S:26

Total S-Trityl-L-cysteine samples: 369
R:354, S:15

Total SB-715992 samples: 770
R:745, S:25

Total VX-680 samples: 365
R:286, S:79

Total ZM-447439 samples: 694
R:628, S:66

/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/MITOSISi/BI-2536
/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/MITOSISi/GSK1070916
/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/MITOSISi/GW843682X
/Volumes/Expansion/Thesis Work/Results/GDSC_DEGs_inhibitors/MITOSISi/Genentech Cpd 10
/Volumes/Expansion/Thesis Work/Results/